<a href="https://colab.research.google.com/github/natdebandi/censo_arg/blob/main/redatam_parser_bivariado_depto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

PARSER CRUCES BIVARIADOS de archivos de REDATAM

frecuencia simple de una variable a nivel departamento

Natalia Debandi


https://redatam.indec.gob.ar/binarg/RpWebEngine.exe/Portal?BASE=CPV2022&lang=ESP

Para usarlo bajar el archivo al repositorio y BORRAR las primeras lineas del encabezadodel excel.

In [5]:
import pandas as pd
import openpyxl
from typing import List, Dict

In [6]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [59]:
def parse_excel_bivariado_depto(file_path: str, row_variable: str, column_variable: str) -> List[Dict]:
    """
    Parsea un archivo Excel con datos de áreas, lugares de nacimiento y sexo registrado al nacer.
    Versión corregida para manejar correctamente los headers de columnas.

    Args:
        file_path: Ruta al archivo Excel
        row_variable: Variable en las filas (ej. "Lugar de nacimiento")
        column_variable: Variable en las columnas (ej. "Sexo registrado al nacer")

    Returns:
        Lista de diccionarios con los datos estructurados
    """
    # Leer el archivo Excel manteniendo los valores originales
    df = pd.read_excel(file_path, header=None, dtype=str)

    current_area = None
    current_depto = None
    results = []
    in_data_section = False
    headers = []

    for index, row in df.iterrows():
        # Convertir la fila a lista limpiando valores NaN/None/espacios
        row_values = [str(cell).strip() if pd.notna(cell) else "" for cell in row]

        # Detectar inicio de nueva área
        if len(row_values) > 1 and row_values[1].startswith('AREA #'):
            current_area = row_values[1]
            current_depto = row_values[2] if len(row_values) > 2 else None
            in_data_section = False
            continue

        # Buscar la fila que contiene los headers (la fila después de las variables)
        if (len(row_values) > 1 and
            row_values[1] == row_variable and
            (len(row_values) > 2 and row_values[2] == column_variable)):

            # La siguiente fila contiene los headers reales
            if index + 1 < len(df):
                next_row = [str(cell).strip() if pd.notna(cell) else "" for cell in df.iloc[index + 1]]
                headers = [h for h in next_row[2:] if h]  # Los headers empiezan desde la columna C (índice 2)
                in_data_section = True
            continue

        # Procesar datos si estamos en la sección correcta
        if in_data_section and row_values[1] and not row_values[1].startswith('Total'):
            row_name = row_values[1]

            # Obtener los valores para cada columna
            values = []
            for i in range(len(headers)):
                col_index = i + 2  # Valores empiezan en columna C (índice 2)
                value = 0
                if col_index < len(row_values) and row_values[col_index].isdigit():
                    value = int(row_values[col_index])
                values.append(value)

            # Crear registros para cada combinación
            for i, header in enumerate(headers):
                if i < len(values):
                    results.append({
                        'area': current_area,
                        'departamento': current_depto,
                        'row_variable': row_variable,
                        'row_value': row_name,
                        'column_variable': column_variable,
                        'column_value': header,
                        'count': values[i]
                    })

    return results

In [60]:
# Cambiar por la ruta del archivo
file_path = '/content/drive/MyDrive/1. Investigacion/1.working documents/Patagonia/redatam_depto/lugar_nac_by_sexo.xlsx'
# Colocar el nombr exacto que figura en el excel
variable_row = 'Lugar de nacimiento'
variable_col = 'Sexo registrado al nacer'

try:
  parsed_data = parse_excel_bivariado_depto(file_path,variable_row,variable_col)
  df_result = pd.DataFrame(parsed_data)
  print(df_result.head())
except Exception as e:
  print(f"Error al procesar el archivo: {e}")


           area departamento         row_variable                 row_value  \
0  AREA # 02007     Comuna 1  Lugar de nacimiento            Esta provincia   
1  AREA # 02007     Comuna 1  Lugar de nacimiento            Esta provincia   
2  AREA # 02007     Comuna 1  Lugar de nacimiento            Esta provincia   
3  AREA # 02007     Comuna 1  Lugar de nacimiento  Otra provincia argentina   
4  AREA # 02007     Comuna 1  Lugar de nacimiento  Otra provincia argentina   

            column_variable       column_value   count  
0  Sexo registrado al nacer   Mujer / Femenino   56749  
1  Sexo registrado al nacer  Varón / Masculino   53879  
2  Sexo registrado al nacer              Total  110628  
3  Sexo registrado al nacer   Mujer / Femenino   29802  
4  Sexo registrado al nacer  Varón / Masculino   26302  


In [61]:
# corrijo el "AREA #"

df_result['area'] = df_result['area'].str.replace('AREA #', '', regex=False)
df_result.head()


,area,departamento,row_variable,row_value,column_variable,column_value,count
0,02007,Comuna 1,Lugar de nacimiento,Esta provincia,Sexo registrado al nacer,Mujer / Femenino,56749
1,02007,Comuna 1,Lugar de nacimiento,Esta provincia,Sexo registrado al nacer,Varón / Masculino,53879
2,02007,Comuna 1,Lugar de nacimiento,Esta provincia,Sexo registrado al nacer,Total,110628
3,02007,Comuna 1,Lugar de nacimiento,Otra provincia argentina,Sexo registrado al nacer,Mujer / Femenino,29802
4,02007,Comuna 1,Lugar de nacimiento,Otra provincia argentina,Sexo registrado al nacer,Varón / Masculino,26302


In [62]:
# Eliminar filas donde 'column_value' es 'Total'
df_result = df_result[df_result['column_value'] != 'Total']

df_result.head()


,area,departamento,row_variable,row_value,column_variable,column_value,count
0,02007,Comuna 1,Lugar de nacimiento,Esta provincia,Sexo registrado al nacer,Mujer / Femenino,56749
1,02007,Comuna 1,Lugar de nacimiento,Esta provincia,Sexo registrado al nacer,Varón / Masculino,53879
3,02007,Comuna 1,Lugar de nacimiento,Otra provincia argentina,Sexo registrado al nacer,Mujer / Femenino,29802
4,02007,Comuna 1,Lugar de nacimiento,Otra provincia argentina,Sexo registrado al nacer,Varón / Masculino,26302
6,02007,Comuna 1,Lugar de nacimiento,Otro país,Sexo registrado al nacer,Mujer / Femenino,29408


In [63]:
###CAMBIAR EL NOMBRE Y PATH
output_file_path = '/content/drive/MyDrive/1. Investigacion/1.working documents/Patagonia/preprocesados/lugar_nac_by_sexo_parsed.csv'
df_result.to_csv(output_file_path, index=False, encoding='utf-8')